In [10]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

In [46]:
def readImage(file):
    #read your file
    img = cv2.imread(file)
    img.shape
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    return thresh

In [8]:
import pandas as pd

In [49]:
def ocr(img):
    ocr = []
    custom_config = r'-l eng --oem 1 --psm 6 '
    d = pytesseract.image_to_data(img, config=custom_config, output_type=Output.DICT)
    df = pd.DataFrame(d)

    df1 = df[(df.conf != '-1') & (df.text != ' ') & (df.text != '')]
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
    for block in sorted_blocks:
        curr = df1[df1['block_num'] == block]
        sel = curr[curr.text.str.len() > 3]
        # sel = curr
        char_w = (sel.width / sel.text.str.len()).mean()
        prev_par, prev_line, prev_left = 0, 0, 0
        text = ''
        for ix, ln in curr.iterrows():
            # add new line when necessary
            if prev_par != ln['par_num']:
                text += '\n'
                prev_par = ln['par_num']
                prev_line = ln['line_num']
                prev_left = 0
            elif prev_line != ln['line_num']:
                text += '\n'
                prev_line = ln['line_num']
                prev_left = 0

            added = 0  # num of spaces that should be added
            if ln['left'] / char_w > prev_left + 1:
                added = int((ln['left']) / char_w) - prev_left
                text += ' ' * added
            text += ln['text'] + ' '
            prev_left += len(ln['text']) + added + 1
        text+='\n'
        ocr.append(text)
    return '\n'.join(ocr)

In [50]:
file=r'./maha.png'
img = readImage(file)
text = ocr(img)
print(text)


                                                       Maharasnva Sate Bectity Ds baton Go tid 
                         Maharashtra             State     Electricity        Distribution          Co.     Ltd. 
                                                         Bill of Supply For: APR-2020 
           Website :www.mahadiscom.in 
           GSTIN  of MSEDCL 27AAECM2933K  1ZB                                                                 HSN  code 27160000 
           BILL NO.(GGN): 000000761530902 
           Consumer  No: 029470759320                                                          Bill Date               20-APR-20 
           SHRI GEOFF  MENDES   ELITA FMENDES                                                  Bill Amount Rs:            590.00 
           FLATA804  PL21 SEC7SIMRAN  RESI KHARGHAR   410210 
           Mobile/Email:                              o7*49/"*"tdubey.  stat@gmail.com         Due Date:               31-MAY-20 
                                      

In [51]:
file=r'./mp2.jpeg'
img = readImage(file)
text = ocr(img)
print(text)


        e 
                i 
    ow    a                oo, 
                                            me 
                  ceacwe   yim  wee 

